# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

In [97]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "alMZBjkqoiGjUdoR8UIDOouaq"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "resource/erm2-nwe9.geojson"
NYC_DATA_TREES = "resource/5rq2-4hqu.geojson"

DB_NAME = "Project_4501"
DB_USER = "cc"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [3]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [119]:
def download_nyc_geojson_data(url, force=False, query=None):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        headers = {'X-App-Token': NYC_DATA_APP_TOKEN}
        params = {
        "$limit": 10000,
        "$offset": 0,
        "$select": "*",
        "$where": query
        }
        response = requests.get(url, headers=headers, params=params)
        
        with open(filename, "w") as f:
            json.dump(response.json(), f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [29]:
def load_and_clean_zipcodes(zipcode_datafile):
    zipcode_data = gpd.read_file(zipcode_datafile)
    zipcode_data = zipcode_data.to_crs(epsg=4326)
    return zipcode_data
    

In [ ]:
def download_and_clean_311_data():
    query_311 = "created_date between '2023-09-30T12:00:00' and '2023-09-30T23:59:59'" #######
    nyc_311_data = download_nyc_geojson_data(f"{BASE_NYC_DATA_URL}{NYC_DATA_311}", force=True, query=query_311)
    nyc_311_data = gpd.read_file(nyc_311_data)
    nyc_311_data = nyc_311_data.to_crs(epsg=4326)
    return nyc_311_data

In [ ]:
def download_and_clean_tree_data():
    """
    Download and clean New York City tree data.

    This function downloads NYC tree data in GeoJSON format from a NYC Open Data,
    reads it into a GeoDataFrame, converts its coordinate reference system to EPSG 4326,
    selects specific columns, renames some for clarity, and removes rows with missing values.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the cleaned NYC tree data.
                          The data includes columns like tree ID, zipcode, species,
                          health, status, and geometry after renaming and dropping
                          missing values.
    """
    nyc_trees_data = download_nyc_geojson_data(f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}", force=True)
    nyc_trees_data = gpd.read_file(nyc_trees_data)
    nyc_trees_data = nyc_trees_data.to_crs(epsg=4326)
    nyc_trees_data = nyc_trees_data[["tree_id", "zipcode", "spc_common", "health", "status", "geometry"]]
    nyc_trees_data = nyc_trees_data.rename(columns={"tree_id": "id_tree", "spc_common": "species"})
    nyc_trees_data = nyc_trees_data.dropna()
    
    return nyc_trees_data

In [ ]:
def load_and_clean_zillow_data(): 
    """
    Load and clean Zillow rental data for New York City.

    This function reads Zillow rental data from a CSV file, filters it for entries
    pertaining to New York City, and selects specific columns relevant to rental
    prices. The columns are then renamed for clarity, and rows with missing values
    are removed to ensure the quality of data.

    Returns:
        DataFrame: A DataFrame containing cleaned Zillow rental data with columns for 
                   Zillow region ID, zipcode, and rental prices for August and September 2023.
    """
    zillow_data = pd.read_csv(ZILLOW_DATA_FILE)
    zillow_data = zillow_data[zillow_data["City"]=="New York"]
    zillow_data = zillow_data[["RegionID", "RegionName", "2023-08-31", "2023-09-30"]]
    zillow_data = zillow_data.rename(columns={"RegionID": "id_zillow", "RegionName": "zipcode", "2023-08-31": "Aug_rent", "2023-09-30": "Sep_rent"})
    zillow_data = zillow_data.dropna()
    
    return zillow_data

In [ ]:
def load_all_data():
    """
    Load and combine multiple datasets into a single collection.

    This function aggregates four different datasets: zipcode data, 311 service request data,
    tree census data, and Zillow rental data, each of which is loaded and cleaned by their
    respective functions. The datasets are primarily focused on New York City information.

    Returns:
        tuple: A tuple containing four cleaned datasets (GeoDataFrames and DataFrame).
               These are: zipcode data, 311 service request data, tree census data,
               and Zillow rental data.
    """
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
TODO
"""

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)